In [ ]:
import cv2
import mediapipe as mp
import time
import random
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import json
import os

GESTURE_LABELS = {
    "draw": "Only index finger open",
    "click": "Index, middle and thumb open",
    "eraser_mode": "Index and middle finger open",
    "draw_mode": "Index, middle, ring fingers open",
    "change_theme": "All fingers open (simplified)",
    "adjust_thickness": "Only thumb open",
    "save": "Thumb and pinky open",
    "load": "Middle, ring, pinky open",
    "clear": "All fingers closed"
}

LIGHTING_CONDITIONS = {
    "bright": (500, 700),
    "normal": (200, 300),
    "dim": (50, 100)
}

SCREEN_HEIGHT, SCREEN_WIDTH = 480, 640
TARGET_COUNT = 5
prev_palm_state = None

def apply_style(style_name):
    """Apply Matplotlib style with fallback to default if style is unavailable."""
    available_styles = plt.style.available
    if style_name in available_styles:
        plt.style.use(style_name)
    else:
        print(f"Style '{style_name}' not available. Falling back to 'default'.")
        plt.style.use('default')

def analyze_gesture(landmarks, hand_label):
    global prev_palm_state

    def is_open(tip, pip): return tip[1] < pip[1]
    def get_point(idx): return landmarks[idx]

    thumb_open = get_point(4)[1] < get_point(2)[1] - 0.03 * SCREEN_HEIGHT
    index_open = is_open(get_point(8), get_point(6))
    middle_open = is_open(get_point(12), get_point(10))
    ring_open = is_open(get_point(16), get_point(14))
    pinky_open = is_open(get_point(20), get_point(18))

    wrist = get_point(0)
    middle_mcp = get_point(9)
    is_palm_facing = wrist[0] < middle_mcp[0] if hand_label == "Right" else wrist[0] > middle_mcp[0]
    all_fingers_open = all([thumb_open, index_open, middle_open, ring_open, pinky_open])

    if index_open and middle_open and not ring_open and not pinky_open and not thumb_open:
        prev_palm_state = is_palm_facing
        return 'eraser_mode'
    if index_open and middle_open and ring_open and not pinky_open and not thumb_open:
        prev_palm_state = is_palm_facing
        return 'draw_mode'
    if all_fingers_open:
        prev_palm_state = is_palm_facing
        return 'change_theme'
    if thumb_open and not index_open and not middle_open and not ring_open and not pinky_open:
        prev_palm_state = is_palm_facing
        return 'adjust_thickness'
    if thumb_open and pinky_open and not index_open and not middle_open and not ring_open:
        prev_palm_state = is_palm_facing
        return 'save'
    if not thumb_open and not index_open and middle_open and ring_open and pinky_open:
        prev_palm_state = is_palm_facing
        return 'load'
    if not any([index_open, middle_open, ring_open, pinky_open, thumb_open]):
        prev_palm_state = is_palm_facing
        return 'clear'
    if index_open and not middle_open and not ring_open and not pinky_open:
        prev_palm_state = is_palm_facing
        return 'draw'
    if index_open and middle_open and not ring_open and not pinky_open and thumb_open:
        prev_palm_state = is_palm_facing
        return 'click'

    prev_palm_state = is_palm_facing
    return 'none'

# Create screenshots folder
SCREENSHOTS_DIR = "screenshots"
if not os.path.exists(SCREENSHOTS_DIR):
    os.makedirs(SCREENSHOTS_DIR)

# Setup
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7, min_tracking_confidence=0.5)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

# Data storage
performance_data = {condition: defaultdict(list) for condition in LIGHTING_CONDITIONS}
gesture_counts = defaultdict(int)
durations = defaultdict(float)
fps_per_gesture = defaultdict(list)

aborted = False

print("Gesture Trainer Started - Press SPACE to record, Q to quit.\n")

for condition in LIGHTING_CONDITIONS:
    print(f"\n=== Testing under {condition.upper()} lighting ({LIGHTING_CONDITIONS[condition][0]}–{LIGHTING_CONDITIONS[condition][1]} lux) ===")
    gesture_counts.clear()
    durations.clear()
    fps_per_gesture.clear()
    last_gesture = None
    prev_time = time.time()

    while True and not aborted:
        available = [g for g in GESTURE_LABELS if gesture_counts[g] < TARGET_COUNT and g != last_gesture]
        if not available:
            break
        current_gesture = random.choice(available)
        last_gesture = current_gesture
        print(f"\n➡️ Show gesture: {current_gesture.upper()} — {GESTURE_LABELS[current_gesture]}")
        start_time = time.time()

        while True:
            ret, frame = cap.read()
            if not ret:
                print("Error: Could not read frame.")
                aborted = True
                break

            frame = cv2.flip(frame, 1)
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(rgb)

            # FPS hesapla
            current_time = time.time()
            fps = 1 / (current_time - prev_time) if (current_time - prev_time) > 0 else 0
            prev_time = current_time
            fps_per_gesture[current_gesture].append(fps)

            # Gesture analiz
            gesture = "No hand"
            if results.multi_hand_landmarks:
                hand = results.multi_hand_landmarks[0]
                mp_draw.draw_landmarks(frame, hand, mp_hands.HAND_CONNECTIONS)
                h, w, _ = frame.shape
                lm = [(pt.x * w, pt.y * h) for pt in hand.landmark]
                hand_label = results.multi_handedness[0].classification[0].label if results.multi_handedness else "Right"
                gesture = analyze_gesture(lm, hand_label)

            cv2.putText(frame, f"Lighting: {condition.upper()}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 200, 0), 2)
            cv2.putText(frame, f"Target: {current_gesture.upper()}", (10, 70),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"Detected: {gesture}", (10, 110),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"FPS: {fps:.2f}", (10, 150),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (200, 255, 200), 2)
            cv2.putText(frame, f"Instruction: {GESTURE_LABELS[current_gesture]}", (10, 190),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.imshow("Gesture Trainer", frame)

            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                aborted = True
                break
            elif key == ord(' ') and gesture == current_gesture:
                response_time = time.time() - start_time
                gesture_counts[gesture] += 1
                # Save screenshot
                screenshot_filename = os.path.join(SCREENSHOTS_DIR, 
                    f"gesture_{gesture}_{gesture_counts[gesture]}_{condition}.png")
                cv2.imwrite(screenshot_filename, frame)
                print(f"📸 Saved screenshot: {screenshot_filename}")
                durations[gesture] += response_time
                performance_data[condition][gesture].append({
                    "sample": gesture_counts[gesture],
                    "response_time": response_time,
                    "fps": fps
                })
                print(f"✅ Captured {gesture_counts[gesture]}/{TARGET_COUNT} for {gesture} (⏱ {response_time:.1f}s, FPS: {fps:.2f})")
                break

    if aborted:
        break

cap.release()
cv2.destroyAllWindows()

# Save performance data to JSON
with open("performance_data.json", "w") as f:
    json.dump(performance_data, f, indent=4)

# Final Summary
if aborted:
    print("\n⚠️ Test aborted by user.\n")
else:
    print("\n✅ All gesture tests complete.\n")

print("=== GESTURE TEST SUMMARY ===")
for condition in LIGHTING_CONDITIONS:
    print(f"\n{condition.upper()} Lighting:")
    for g in GESTURE_LABELS:
        samples = performance_data[condition][g]
        avg_fps = np.mean([s["fps"] for s in samples]) if samples else 0
        avg_time = np.mean([s["response_time"] for s in samples]) if samples else 0
        print(f"{g:15} | {len(samples):2d}/5 samples | Avg Time: {avg_time:.1f}s | Avg FPS: {avg_fps:.2f}")

# Plotting Results
all_gestures = list(GESTURE_LABELS.keys())

# Plot 1: Bar Plot for Average Response Time by Lighting Condition
apply_style('seaborn-v0_8')
plt.figure(figsize=(12, 6))
bar_width = 0.25
x = np.arange(len(all_gestures))
for i, condition in enumerate(LIGHTING_CONDITIONS):
    avg_times = [np.mean([s["response_time"] for s in performance_data[condition][g]]) if performance_data[condition][g] else 0 for g in all_gestures]
    plt.bar(x + i * bar_width, avg_times, bar_width, label=condition.capitalize())
plt.title("Average Response Time by Gesture and Lighting", fontsize=14)
plt.xlabel("Gestures", fontsize=12)
plt.ylabel("Response Time (s)", fontsize=12)
plt.xticks(x + bar_width, all_gestures, rotation=45, fontsize=10)
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig("response_time_bar.png")
plt.close()

# Plot 2: Pie Chart for Sample Counts per Lighting Condition
apply_style('default')
plt.figure(figsize=(10, 8))
colors = plt.cm.Paired(np.linspace(0, 1, len(LIGHTING_CONDITIONS)))
sample_counts = [sum(len(performance_data[condition][g]) for g in all_gestures) for condition in LIGHTING_CONDITIONS]
plt.pie(sample_counts, labels=[c.capitalize() for c in LIGHTING_CONDITIONS], colors=colors, autopct='%1.1f%%', startangle=140)
plt.title("Distribution of Samples by Lighting Condition", fontsize=14)
plt.tight_layout()
plt.savefig("sample_counts_pie.png")
plt.close()

# Plot 3: Stacked Bar Plot for FPS by Gesture and Lighting
apply_style('ggplot')
plt.figure(figsize=(12, 6))
x = np.arange(len(all_gestures))
bottom = np.zeros(len(all_gestures))
for condition in LIGHTING_CONDITIONS:
    avg_fps = [np.mean([s["fps"] for s in performance_data[condition][g]]) if performance_data[condition][g] else 0 for g in all_gestures]
    plt.bar(x, avg_fps, bottom=bottom, label=condition.capitalize())
    bottom += np.array(avg_fps)
plt.title("Stacked FPS by Gesture and Lighting", fontsize=14)
plt.xlabel("Gestures", fontsize=12)
plt.ylabel("FPS", fontsize=12)
plt.xticks(x, all_gestures, rotation=45, fontsize=10)
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig("fps_stacked_bar.png")
plt.close()

# Plot 4: Line Plot for Response Time Trends
apply_style('bmh')
plt.figure(figsize=(12, 6))
for condition in LIGHTING_CONDITIONS:
    avg_times = [np.mean([s["response_time"] for s in performance_data[condition][g]]) if performance_data[condition][g] else 0 for g in all_gestures]
    plt.plot(all_gestures, avg_times, marker='o', linestyle='-', linewidth=2, label=condition.capitalize())
plt.title("Response Time Trends by Gesture and Lighting", fontsize=14)
plt.ylabel("Response Time (s)", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.xticks(rotation=45, fontsize=10)
plt.legend()
plt.tight_layout()
plt.savefig("response_time_line.png")
plt.close()  

SyntaxError: invalid syntax (2659896851.py, line 7)